In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 

In [2]:
# 2014 
df = pd.read_csv(r'I:\T2040\soundcast_2014\outputs\daysim\_trip.tsv', sep='\t')
hh = pd.read_csv(r'I:\T2040\soundcast_2014\outputs\daysim\_household.tsv', sep='\t')
parcels = pd.read_csv(r'I:\T2040\soundcast_2014\inputs\accessibility\parcels_urbansim.txt', sep = ' ')

rgc_pcl = pd.read_csv(r'I:\T2040\soundcast_2014\scripts\summarize\inputs\parcels_in_urbcens.csv')
equity_taz = pd.read_csv(r'I:\T2040\soundcast_2014\scripts\summarize\inputs\special_needs_taz.csv')

housing_cost_pcl= pd.read_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\parcel_housing_costs_2014_lodes.csv')

In [3]:
# AAA Calculations availavble for small, medium, large Sedans (and average of these), SUV, Minivan

# Analysis of survey could be used to allocate vehicle type distributions?

# http://publicaffairsresources.aaa.biz/wp-content/uploads/2016/03/2016-YDC-Brochure.pdf
# results are averages for sedans

# Annualalize costs as 262
annual_factor = 262

# Costs by miles
# Operating costs total (gas + maintenance + tires) using model estimates for this in the travcost field
depreciation_per_mile = 3759.0/15000    # depreciation by mile

# Annual costs
insurance = 1222    #(full-coverage insurance, license, registration, taxes, depreciation @ 15,000 miles per year), finance charge
taxes = 687    # license, registration, taxes
finance = 683    # load financing

annual_fixed_costs = insurance+taxes+finance

In [4]:
# to calculate activity durations for parking cost:
day_minutes = 1440
minutes_hr = 60.0
cents_dollar = 100

In [5]:
# get total travel cost (based on vmt) for household records
hh_mile_costs = df.groupby('hhno').sum()[['travcost']]
hh_mile_costs = hh_mile_costs.reset_index()

hh = pd.merge(hh, hh_mile_costs, on='hhno', how='left')

# Some households have 0 travel cost
hh['travcost'] = hh['travcost'].fillna(0)

# Total vehicle miles traveled per household to calculate mile-based costs
driver_trips = df[df['dorp'] == 1]

# Get total travel distance 
hh_travdist = df.groupby('hhno').sum()[['travdist']]
hh_travdist = hh_travdist.reset_index()

hh = pd.merge(hh, hh_travdist, on='hhno', how='left')

# annual operating costs
driver_trips['operating_cost_drivers'] = driver_trips['travcost']*annual_factor
driver_trips['travdist'] = driver_trips['travdist'].fillna(0)
driver_trips['depreciation'] = driver_trips['travdist']*depreciation_per_mile*annual_factor

#calculating parking costs based on activity duration
# the results look weird - add in parking cost later
driver_trips = pd.merge(driver_trips, parcels[['PARCELID','PPRICHRP']], left_on='dpcl', right_on = 'PARCELID')
# if a trip spans midnight you need to adjust across the day - like this:
driver_trips['duration'] = np.where(driver_trips['endacttm']<driver_trips['arrtm'], day_minutes-driver_trips['arrtm'] +driver_trips['endacttm'], driver_trips['endacttm']-driver_trips['arrtm'])/minutes_hr
driver_trips['parking_cost'] = (driver_trips['duration']*driver_trips['PPRICHRP']*annual_factor)/cents_dollar



driver_trips_hh = driver_trips.groupby('hhno').sum()[['operating_cost_drivers','depreciation','parking_cost']].reset_index()
hh = pd.merge(hh, driver_trips_hh, on = 'hhno')

# fixed costs as function of number of vehicles owned
hh['veh_insurance'] = hh['hhvehs']*insurance
hh['veh_taxes'] = hh['hhvehs']*taxes
hh['veh_finance'] = hh['hhvehs']*finance

hh['annual_auto_costs'] = hh['operating_cost_drivers']+hh['depreciation']+hh['veh_insurance']+hh['veh_taxes']+hh['veh_finance']
hh['annual_auto_costs']=hh['annual_auto_costs'].fillna(0)
# Need to add per mile depreciation costs

# Calculate transit cost
transit_trips = df[df['mode'] == 6]

hh_travdist_transit_cost = transit_trips.groupby('hhno').sum()[['travcost']]
hh_travdist_transit_cost['transit_cost'] = hh_travdist_transit_cost['travcost']
hh_travdist_transit_cost = hh_travdist_transit_cost.reset_index()
hh_travdist_transit_cost['annual_transit_cost'] = hh_travdist_transit_cost['transit_cost']*annual_factor

hh = pd.merge(hh, hh_travdist_transit_cost[['hhno','annual_transit_cost']], on='hhno', how='left')
hh['annual_transit_cost'] = hh['annual_transit_cost'].fillna(0)

hh['total_cost']= hh['annual_transit_cost']+hh['annual_auto_costs']
hh = hh.loc[hh['hhincome']>hh['total_cost']]
hh['percent_transport_cost'] = hh['total_cost']/hh['hhincome']
hh['percent_transport_cost'] =hh['percent_transport_cost'].fillna(0)

print hh['total_cost'].mean()
print hh['veh_insurance'].mean()
print hh['veh_taxes'].mean()
print hh['operating_cost_drivers'].mean()
print hh['depreciation'].mean()
print hh['annual_transit_cost'].mean()
print hh['parking_cost'].mean()
print hh['percent_transport_cost'].mean()

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

11030.5148324
2473.99768732
1390.86449361
2577.46964864
3129.28900109
76.1277002655
442.390041972
0.175471651091


In [6]:
hh_cost_taz = hh.groupby('hhtaz').mean()
hh_cost_taz.to_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\hh_costs_taz.csv')

In [7]:
# get costs by center
hh_cost_rgc = pd.merge(hh, rgc_pcl, left_on= 'hhparcel', right_on = 'hhparcel', how= 'left')
hh_cost_rgc=hh_cost_rgc.fillna('All Not Center')
hh_cost_rgc = pd.merge(hh_cost_rgc, housing_cost_pcl, left_on = 'hhparcel', right_on='parcelid')
hh_cost_rgc = hh_cost_rgc.groupby('NAME').mean()
hh_cost_rgc['year'] = 'Now'

#hh_cost_rgc.to_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\hh_costs_taz.csv'

hh_cost_equity = pd.merge(hh, equity_taz, left_on= 'hhtaz', right_on = 'TAZ', how= 'left')
hh_cost_equity=hh_cost_equity.fillna('Not Majority Minority')
hh_cost_equity = hh_cost_equity.groupby('Minority').mean()
hh_cost_equity['year'] = 'Now'


In [8]:
hh_cost_rgc

,hhno,fraction_with_jobs_outside,hhsize,hhvehs,hhwkrs,hhftw,hhptw,hhret,hhoad,hhuni,...,emp_ndx,block_size,intersection_density,avg_block_perimeter_meters,h_cost,median_smoc,median_gross_rent,pct_owner_occupied_hu,pct_renter_occupied_hu,year
NAME,,,,,,,,,,,,,,,,,,,,,
All Not Center,7.678571e+05,0.036870,2.636955,2.024054,-1.0,1.031703,0.236648,0.253704,0.382612,0.114329,...,89.602035,29.110242,200.149281,1199.744750,1763.371823,2051.352176,1308.020197,63.017466,36.985672,Now
Auburn,1.101903e+06,0.062508,2.833333,2.179894,-1.0,1.037037,0.264550,0.322751,0.447090,0.156085,...,87.243386,101.595238,117.145503,1672.714286,1636.144172,1758.532915,1192.615108,74.931217,25.074074,Now
Bellevue,1.260876e+06,0.043153,2.751701,2.241497,-1.0,1.112245,0.197279,0.343537,0.496599,0.095238,...,88.037415,42.282313,143.482993,1423.894558,2012.510204,2219.462585,1286.108108,75.714286,24.285714,Now
Bothell Canyon Park,2.881025e+05,0.009348,2.600000,1.765217,-1.0,0.926087,0.286957,0.247826,0.430435,0.173913,...,91.200000,7.243478,317.173913,697.604348,1706.899123,2095.860335,1242.719577,59.147826,40.852174,Now
Bremerton,1.131398e+06,0.134788,2.941379,2.089655,-1.0,1.091379,0.255172,0.189655,0.422414,0.127586,...,88.143103,25.143103,160.262069,1167.315517,1576.829310,1791.664921,1148.909789,66.360345,33.639655,Now
Burien,1.382076e+06,0.030111,2.634799,2.124283,-1.0,1.009560,0.263862,0.319312,0.357553,0.074570,...,87.151052,58.734226,124.646272,1781.877629,1732.841300,1826.122371,1289.912929,82.514340,17.485660,Now
Everett,4.173178e+05,0.013672,2.790857,2.161143,-1.0,1.145143,0.248000,0.209143,0.388571,0.153143,...,90.410286,12.104000,258.397714,898.377143,2037.626286,2266.376009,1529.642670,69.014857,30.985143,Now
Federal Way,1.334579e+06,0.054479,2.687500,2.062500,-1.0,0.937500,0.125000,0.520833,0.291667,0.083333,...,87.312500,121.895833,109.583333,2407.937500,1898.511111,1966.533333,1441.791667,76.854167,23.145833,Now
Kent,1.190133e+06,0.093346,2.774278,2.131234,-1.0,1.070866,0.228346,0.291339,0.412073,0.086614,...,88.422572,23.236220,153.761155,1156.422572,1547.238845,1812.769029,1180.064972,63.947507,36.052493,Now


## Scenario comparisons

In [9]:

df_compare = pd.read_csv(r'I:\T2040\soundcast_2040_constrained\outputs\daysim\_trip.tsv', sep='\t')
hh_compare = pd.read_csv(r'I:\T2040\soundcast_2040_constrained\outputs\daysim\_household.tsv', sep='\t')
parcels_compare = pd.read_csv(r'I:\T2040\soundcast_2040_constrained\inputs\accessibility\parcels_urbansim.txt', sep = ' ')



In [10]:

# get total travel cost (based on vmt) for household records
hh_compare_mile_costs = df_compare.groupby('hhno').sum()[['travcost']]
hh_compare_mile_costs = hh_compare_mile_costs.reset_index()

hh_compare = pd.merge(hh_compare, hh_compare_mile_costs, on='hhno', how='left')

# Some households have 0 travel cost
hh_compare['travcost'] = hh_compare['travcost'].fillna(0)

# Total vehicle miles traveled per household to calculate mile-based costs
driver_trips = df_compare[df_compare['dorp'] == 1]

# Get total travel distance 
hh_compare_travdist = df_compare.groupby('hhno').sum()[['travdist']]
hh_compare_travdist = hh_compare_travdist.reset_index()

hh_compare = pd.merge(hh_compare, hh_compare_travdist, on='hhno', how='left')

# annual operating costs
driver_trips['operating_cost_drivers'] = driver_trips['travcost']*annual_factor
driver_trips['travdist'] = driver_trips['travdist'].fillna(0)
driver_trips['depreciation'] = driver_trips['travdist']*depreciation_per_mile*annual_factor

#calculating parking costs based on activity duration
# the results look weird - add in parking cost later
driver_trips = pd.merge(driver_trips, parcels_compare[['PARCELID','PPRICHRP']], left_on='dpcl', right_on = 'PARCELID')
# if a trip spans midnight you need to adjust across the day - like this:
driver_trips['duration'] = np.where(driver_trips['endacttm']<driver_trips['arrtm'], day_minutes-driver_trips['arrtm'] +driver_trips['endacttm'], driver_trips['endacttm']-driver_trips['arrtm'])/minutes_hr
driver_trips['parking_cost'] = (driver_trips['duration']*driver_trips['PPRICHRP']*annual_factor)/cents_dollar



driver_trips_hh_compare = driver_trips.groupby('hhno').sum()[['operating_cost_drivers','depreciation','parking_cost']].reset_index()
hh_compare = pd.merge(hh_compare, driver_trips_hh_compare, on = 'hhno')

# fixed costs as function of number of vehicles owned
hh_compare['veh_insurance'] = hh_compare['hhvehs']*insurance
hh_compare['veh_taxes'] = hh_compare['hhvehs']*taxes
hh_compare['veh_finance'] = hh_compare['hhvehs']*finance

hh_compare['annual_auto_costs'] = hh_compare['operating_cost_drivers']+hh_compare['depreciation']+hh_compare['veh_insurance']+hh_compare['veh_taxes']+hh_compare['veh_finance']
hh_compare['annual_auto_costs']=hh_compare['annual_auto_costs'].fillna(0)
# Need to add per mile depreciation costs

# Calculate transit cost
transit_trips = df[df['mode'] == 6]

hh_compare_travdist_transit_cost = transit_trips.groupby('hhno').sum()[['travcost']]
hh_compare_travdist_transit_cost['transit_cost'] = hh_compare_travdist_transit_cost['travcost']
hh_compare_travdist_transit_cost = hh_compare_travdist_transit_cost.reset_index()
hh_compare_travdist_transit_cost['annual_transit_cost'] = hh_compare_travdist_transit_cost['transit_cost']*annual_factor

hh_compare = pd.merge(hh_compare, hh_compare_travdist_transit_cost[['hhno','annual_transit_cost']], on='hhno', how='left')
hh_compare['annual_transit_cost'] = hh_compare['annual_transit_cost'].fillna(0)

hh_compare['total_cost']= hh_compare['annual_transit_cost']+hh_compare['annual_auto_costs']
hh_compare = hh_compare.loc[hh_compare['hhincome']>hh_compare['total_cost']]
hh_compare['percent_transport_cost'] = hh_compare['total_cost']/hh_compare['hhincome']
hh_compare['percent_transport_cost'] =hh_compare['percent_transport_cost'].fillna(0)

print hh_compare['total_cost'].mean()
print hh_compare['veh_insurance'].mean()
print hh_compare['veh_taxes'].mean()
print hh_compare['operating_cost_drivers'].mean()
print hh_compare['depreciation'].mean()
print hh_compare['annual_transit_cost'].mean()
print hh_compare['parking_cost'].mean()
print hh_compare['percent_transport_cost'].mean()

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

11179.6526341
2406.57972401
1352.96257806
3362.26590762
2654.4620296
58.2973281076
3885.76582244
0.179280046403


In [11]:
hh_cost_compare_rgc = pd.merge(hh_compare, rgc_pcl, left_on= 'hhparcel', right_on = 'hhparcel', how= 'left')
hh_cost_compare_rgc=hh_cost_compare_rgc.fillna('All Not Center')
hh_cost_compare_rgc = pd.merge(hh_cost_compare_rgc, housing_cost_pcl, left_on = 'hhparcel', right_on='parcelid')
hh_cost_compare_rgc = hh_cost_compare_rgc.groupby('NAME').mean()
hh_cost_compare_rgc['year'] = 'Plan 2040'

In [12]:
hh_cost_compare_equity = pd.merge(hh_compare, equity_taz, left_on= 'hhtaz', right_on = 'TAZ', how= 'left')
hh_cost_compare_equity=hh_cost_compare_equity.fillna('Not Majority Minority')
hh_cost_compare_equity = hh_cost_compare_equity.groupby('Minority').mean()
hh_cost_compare_equity['year'] = 'Plan 2040'

In [14]:
hh_cost_compare_rgc = pd.merge(hh_compare, rgc_pcl, left_on= 'hhparcel', right_on = 'hhparcel', how= 'left')
hh_cost_compare_rgc=hh_cost_compare_rgc.fillna('All Not Center')
hh_cost_compare_rgc = pd.merge(hh_cost_compare_rgc, housing_cost_pcl, left_on = 'hhparcel', right_on='parcelid')
hh_cost_compare_rgc = hh_cost_compare_rgc.groupby('NAME').mean()
hh_cost_compare_rgc['year'] = 'Plan 2040'

In [16]:
# put the comparisons together
hh_cost_rgc = hh_cost_rgc.append(hh_cost_compare_rgc)
hh_cost_equity = hh_cost_equity.append(hh_cost_compare_equity)
hh_cost_rgc['Annual Housing Costs'] = hh_cost_rgc['h_cost']*12

In [17]:
hh_cost_rgc

,hhno,fraction_with_jobs_outside,hhsize,hhvehs,hhwkrs,hhftw,hhptw,hhret,hhoad,hhuni,...,block_size,intersection_density,avg_block_perimeter_meters,h_cost,median_smoc,median_gross_rent,pct_owner_occupied_hu,pct_renter_occupied_hu,year,Annual Housing Costs
NAME,,,,,,,,,,,,,,,,,,,,,
All Not Center,7.678571e+05,0.036870,2.636955,2.024054,-1.0,1.031703,0.236648,0.253704,0.382612,0.114329,...,29.110242,200.149281,1199.744750,1763.371823,2051.352176,1308.020197,63.017466,36.985672,Now,21160.461882
Auburn,1.101903e+06,0.062508,2.833333,2.179894,-1.0,1.037037,0.264550,0.322751,0.447090,0.156085,...,101.595238,117.145503,1672.714286,1636.144172,1758.532915,1192.615108,74.931217,25.074074,Now,19633.730061
Bellevue,1.260876e+06,0.043153,2.751701,2.241497,-1.0,1.112245,0.197279,0.343537,0.496599,0.095238,...,42.282313,143.482993,1423.894558,2012.510204,2219.462585,1286.108108,75.714286,24.285714,Now,24150.122449
Bothell Canyon Park,2.881025e+05,0.009348,2.600000,1.765217,-1.0,0.926087,0.286957,0.247826,0.430435,0.173913,...,7.243478,317.173913,697.604348,1706.899123,2095.860335,1242.719577,59.147826,40.852174,Now,20482.789474
Bremerton,1.131398e+06,0.134788,2.941379,2.089655,-1.0,1.091379,0.255172,0.189655,0.422414,0.127586,...,25.143103,160.262069,1167.315517,1576.829310,1791.664921,1148.909789,66.360345,33.639655,Now,18921.951724
Burien,1.382076e+06,0.030111,2.634799,2.124283,-1.0,1.009560,0.263862,0.319312,0.357553,0.074570,...,58.734226,124.646272,1781.877629,1732.841300,1826.122371,1289.912929,82.514340,17.485660,Now,20794.095602
Everett,4.173178e+05,0.013672,2.790857,2.161143,-1.0,1.145143,0.248000,0.209143,0.388571,0.153143,...,12.104000,258.397714,898.377143,2037.626286,2266.376009,1529.642670,69.014857,30.985143,Now,24451.515429
Federal Way,1.334579e+06,0.054479,2.687500,2.062500,-1.0,0.937500,0.125000,0.520833,0.291667,0.083333,...,121.895833,109.583333,2407.937500,1898.511111,1966.533333,1441.791667,76.854167,23.145833,Now,22782.133333
Kent,1.190133e+06,0.093346,2.774278,2.131234,-1.0,1.070866,0.228346,0.291339,0.412073,0.086614,...,23.236220,153.761155,1156.422572,1547.238845,1812.769029,1180.064972,63.947507,36.052493,Now,18566.866142


In [18]:
hh_cost_rgc.to_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\transport_costs_rgc.csv')
hh_cost_equity.to_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\transport_costs_equity.csv')